In [1]:
%matplotlib
import numpy as np
import h5py
import os, sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import image as mpimg
import yaml
import argparse
import xml.etree.ElementTree as ET

sys.path.append('../')
import utils
import settings
plt.style.use('../spectrum.mplstyle')
# plt.ion()

Using matplotlib backend: Qt5Agg


In [2]:
def multi_exp_fit(param_data_object, t, corr, result_type="ct"):
    n = [2.0,1.0,1.0,1.0]
    param_info = param_data_object['param_info'][corr].attrs
    for i in range(0,4):
        if f'N{i+2}' in param_info.keys():
            n[i] = float(param_info[f'N{i+2}'])
    param_data = param_data_object['params']['Values']
    param_samplings = [param_data[f'<MCObservable><Info>{fit_param}<|Info><|MCObservable>'][()] for fit_param in param_info['FitParams']]
#     print(corr,param_samplings)
    function_vals = []
    function_errs = []
    for tval in t:
        if result_type=="ct":
            function_samplings = utils.multi_exp_func(tval, param_samplings[0], param_samplings[1], param_samplings[2], 
                   param_samplings[3], param_samplings[4], param_samplings[5], 
                   param_samplings[6], param_samplings[7],n[0],n[1], n[2],n[3])
        else:
            function_samplings = utils.multi_exp_func_eff(tval, param_samplings[0], param_samplings[1], param_samplings[2], 
                   param_samplings[3], param_samplings[4], param_samplings[5], 
                   param_samplings[6], param_samplings[7],n[0],n[1], n[2],n[3])
        function_vals.append(function_samplings[0])
        function_errs.append(utils.bootstrap_error_by_array(function_samplings))
    return np.array(function_vals),np.array(function_errs)

In [3]:
def get_fit_range(param_data_object, corr):
    param_info = param_data_object['param_info'][corr].attrs
    return param_info["FinalTmin"], param_info["FinalTmax"]

In [4]:
data_files = {
    "single_hadrons.hdf5":'single_hadrons',
    "rotated_correlators.hdf5":'rotated_correlators'
}
param_files = {
    "t=[2,17],gap=1.0": "param_samplings_isosinglet_strange_fermionic_multi_rebin10-17.hdf5",
    "t=[2,19],gap=1.0": "param_samplings_isosinglet_strange_fermionic_multi_rebin10-19-3.hdf5",
    "t=[2,21],gap=1.0": "param_samplings_isosinglet_strange_fermionic_multi_rebin10-21.hdf5",
    "t=[2,23],gap=1.0": "param_samplings_isosinglet_strange_fermionic_multi_rebin10-23.hdf5",
    "t=[2,25],gap=1.0": "param_samplings_isosinglet_strange_fermionic_multi_rebin10-25.hdf5",
#     "t=[2,19],gap=1.0": "param_samplings_isosinglet_strange_fermionic_multi_rebin10-19.hdf5",
#     "t=[2,19]*,gap=1.0": "param_samplings_isosinglet_strange_fermionic_multi_rebin10-19-2.hdf5",
}
johns_data_file = "nucleon_d0_emass_1_0_errs.dat"
n_corr = 'isodoublet S=0 P=(0,0,0) G1g N[SS0] 0'

available_corrs = []
for file in param_files:
    param_samplings = h5py.File(param_files[file],"r")
    available_corrs += list( param_samplings['param_info'].keys() )
    param_samplings.close()

In [5]:
available_corrs = list(set(available_corrs))
# available_corrs = [n_corr]

In [6]:
# f = plt.figure(facecolor="white")
f, (ax1, ax2) = plt.subplots(1,2)
f.set_figwidth(16)
f.set_figheight(8)
for corr in available_corrs:
    print(corr)
#     plt.clf()
    ax1.clear()
    ax2.clear()
    f.suptitle(corr.replace("_"," "))
    i=0
    for data_file in data_files.keys():
        hdf5_data = h5py.File(data_file,"r")
#         plt.subplot(1,2,1)
        this_t, this_corr, this_err = utils.collectCorrEstimates(hdf5_data,corr,tag=data_files[data_file])
        if this_t:
            ax1.errorbar( this_t, this_corr,this_err, label="data",  color=settings.colors[i], marker=settings.markers[i], linewidth=0.0, elinewidth=2.0, capsize=5.0 )
#         plt.subplot(1,2,2)
        this_t, this_corr, this_err = utils.collectEnergyEstimates(hdf5_data,corr,tag=data_files[data_file])
        if this_t:
            ax2.errorbar( this_t, this_corr,this_err, label="data",  color=settings.colors[i], marker=settings.markers[i], linewidth=0.0, elinewidth=2.0, capsize=5.0 )
        hdf5_data.close()
    i=1
    for dataset in param_files.keys():
        hdf5_data = h5py.File(param_files[dataset],"r")
        tmin, tmax = get_fit_range(hdf5_data,corr)
        ax1.axvspan(tmin, tmax, ymin=1.0-(i)/len(param_files),ymax=1.0-(i-1)/len(param_files), alpha=0.2,color=settings.colors[i])
        ax2.axvspan(tmin, tmax, ymin=1.0-(i)/len(param_files),ymax=1.0-(i-1)/len(param_files), alpha=0.2,color=settings.colors[i])
        t = np.linspace(2.0,25.0,1000)
        itmin = np.where(t>=tmin)[0][0]
        itmax = np.where(t>=tmax)[0][0]
        fit_vals = np.array([])
        try:
            fit_vals, fit_errs = multi_exp_fit(hdf5_data,t,corr,"ct")
        except Exception as error:
            print(error) #pass
#         plt.subplot(1,2,1)
        data_label = f"{dataset}, {round(hdf5_data['param_info'][corr].attrs['ChiSquarePerDof'],2)}"
        if fit_vals.any():
            ax1.plot(t,fit_vals,ls="--",color=settings.colors[i])
            ax1.plot(t,fit_vals+fit_errs,ls="--",color=settings.colors[i])
            ax1.plot(t,fit_vals-fit_errs,ls="--",color=settings.colors[i])
            ax1.plot(t[itmin:itmax],fit_vals[itmin:itmax],label=data_label,color=settings.colors[i])
            ax1.plot(t[itmin:itmax],fit_vals[itmin:itmax]+fit_errs[itmin:itmax],color=settings.colors[i])
            ax1.plot(t[itmin:itmax],fit_vals[itmin:itmax]-fit_errs[itmin:itmax],color=settings.colors[i])
        ax1.set_xlabel("$t$")
        ax1.set_ylabel("$C(t)$")
            
        fit_vals = np.array([])
        try:
            fit_vals, fit_errs = multi_exp_fit(hdf5_data,t,corr,"eff")
        except Exception as error:
            print(error) #pass
#         plt.subplot(1,2,2)
        if fit_vals.any():
            ax2.plot(t,fit_vals,ls="--",color=settings.colors[i])
            ax2.plot(t,fit_vals+fit_errs,ls="--",color=settings.colors[i])
            ax2.plot(t,fit_vals-fit_errs,ls="--",color=settings.colors[i])
            ax2.plot(t[itmin:itmax],fit_vals[itmin:itmax],label=data_label,color=settings.colors[i])
            ax2.plot(t[itmin:itmax],fit_vals[itmin:itmax]+fit_errs[itmin:itmax],color=settings.colors[i])
            ax2.plot(t[itmin:itmax],fit_vals[itmin:itmax]-fit_errs[itmin:itmax],color=settings.colors[i])
#         plt.legend()
        ax2.set_xlabel("$t$")
        ax2.set_ylabel("$E_{eff}(t)$")
        ax2.legend()
        hdf5_data.close()
        i+=1
        
    if corr==n_corr:
        john_data = pd.read_csv(johns_data_file, header=None,skiprows=6,delimiter="   ")
        ax2.errorbar(john_data[0],john_data[1],john_data[2],label="john",color=settings.colors[i], marker=settings.markers[i], linewidth=0.0, elinewidth=2.0, capsize=5.0)
        ax2.legend()
        
    plt.tight_layout()
    plt.savefig(corr+".png")

isosinglet S=-1 PSQ=3 G ROT 8
isosinglet S=-1 P=(0,0,0) G1u ROT 0
isosinglet S=-1 PSQ=3 F1 ROT 5


..\utils.py:22: RuntimeWarning: invalid value encountered in log
  return t[:-1]+0.5*(t[1]-t[0]), np.log( C[:-1]/C[1:] )


isosinglet S=-1 PSQ=3 G ROT 4
isosinglet S=-1 P=(0,0,0) G1u ROT 4
isosinglet S=-1 PSQ=3 F2 ROT 2
isosinglet S=-1 PSQ=2 G ROT 5
isodoublet S=-1 PSQ=4 A2 k[SS1] 0
isosinglet S=-1 PSQ=3 F2 ROT 3
isotriplet S=0 PSQ=1 A2m P[SS1] 0
isosinglet S=-1 PSQ=2 G ROT 4
isosinglet S=-1 PSQ=3 G ROT 9
isosinglet S=-1 PSQ=3 F2 ROT 5


..\utils.py:22: RuntimeWarning: invalid value encountered in log
  return t[:-1]+0.5*(t[1]-t[0]), np.log( C[:-1]/C[1:] )


isodoublet S=0 PSQ=3 G N[SS0] 0
isosinglet S=-1 PSQ=3 F2 ROT 4
isosinglet S=-1 PSQ=1 G2 ROT 5
isosinglet S=-1 PSQ=3 F1 ROT 2
isosinglet S=-1 PSQ=1 G2 ROT 3
isotriplet S=-1 PSQ=3 G S[SS0] 0
isotriplet S=-1 PSQ=2 G S[SS0] 0
isosinglet S=-1 P=(0,0,0) G1g ROT 0
isotriplet S=-1 P=(0,0,0) G1g S[SS0] 0
isosinglet S=-1 PSQ=1 G1 ROT 1
isosinglet S=-1 PSQ=2 G ROT 7
isosinglet S=-1 PSQ=2 G ROT 12


...

isosinglet S=-1 PSQ=3 F2 ROT 0
isosinglet S=-1 PSQ=1 G1 ROT 2
isodoublet S=0 PSQ=1 G1 N[SS0] 0
isodoublet S=0 P=(0,0,0) G1g N[SS0] 0


<ipython-input-6-c09bfd0d1b17>:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  john_data = pd.read_csv(johns_data_file, header=None,skiprows=6,delimiter="   ")


isosinglet S=-1 PSQ=2 G ROT 11


..\utils.py:22: RuntimeWarning: invalid value encountered in log
  return t[:-1]+0.5*(t[1]-t[0]), np.log( C[:-1]/C[1:] )


isosinglet S=-1 PSQ=1 G2 ROT 0
isosinglet S=-1 PSQ=2 G ROT 1
isosinglet S=-1 PSQ=3 G ROT 6
isosinglet S=-1 PSQ=3 G ROT 7
isosinglet S=-1 P=(0,0,0) G1g ROT 2


In [7]:
hdf5_data = h5py.File("param_samplings_isosinglet_strange_fermionic_multi_rebin10.hdf5","r")
print(hdf5_data['params']['Values'].keys() )
hdf5_data.close()

<KeysViewHDF5 ['<MCObservable><Info>1-1000G1g-L[SS0]-0T2-25 112 n re<|Info><|MCObservable>', '<MCObservable><Info>1-1000G1g-L[SS0]-0T2-25 12 n re<|Info><|MCObservable>', '<MCObservable><Info>1-1000G1g-L[SS0]-0T2-25 212 n re<|Info><|MCObservable>', '<MCObservable><Info>1-1000G1g-L[SS0]-0T2-25 312 n re<|Info><|MCObservable>', '<MCObservable><Info>1-1000G1g-L[SS0]-0T2-25 412 n re<|Info><|MCObservable>', '<MCObservable><Info>1-1000G1g-L[SS0]-0T2-25 512 n re<|Info><|MCObservable>', '<MCObservable><Info>1-1000G1g-L[SS0]-0T2-25 612 n re<|Info><|MCObservable>', '<MCObservable><Info>1-1000G1g-L[SS0]-0T2-25 712 n re<|Info><|MCObservable>', '<MCObservable><Info>1-1000G1g-ROT-0T2-25 112 n re<|Info><|MCObservable>', '<MCObservable><Info>1-1000G1g-ROT-0T2-25 12 n re<|Info><|MCObservable>', '<MCObservable><Info>1-1000G1g-ROT-0T2-25 212 n re<|Info><|MCObservable>', '<MCObservable><Info>1-1000G1g-ROT-0T2-25 312 n re<|Info><|MCObservable>', '<MCObservable><Info>1-1000G1g-ROT-0T2-25 412 n re<|Info><|MCOb

In [8]:

hdf5_data.close()